<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/GRPO%5CLlama3_1_(8B)_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

[unsloth](https://unsloth.ai/)

[join our discord](https://discord.gg/unsloth)

[Documentation](https://docs.unsloth.ai/)

Join Discord if you need help +  <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i>


To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance to train reasoning model.** GRPO notebook is inspired by [@shxf0072](https://x.com/shxf0072/status/1886085377146180091), [@Teknium1](https://x.com/Teknium1/status/1885077369142337550), [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
#!pip install --upgrade pillow
!pip install Pillow==11.1.0
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    model_name = "meta-llama/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-04 03:28:34 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.32 GB. Also swap space = 2 GB.
WARNING 03-04 03:28:37 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-04 03:29:01 config.py:549] This model supp

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 03-04 03:30:25 model_runner.py:1562] Graph capturing finished in 39 secs, took 0.51 GiB
INFO 03-04 03:30:25 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 53.19 seconds


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 48,627,712


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
To find out the total amount Mr. Benson paid for the concert tickets, we need to first find the cost of the first 10 tickets and the discount for the remaining 2 tickets.

The first 10 tickets have no discount applied as they are the base price of $40. So, the cost for the first 10 tickets is 10 * $40 = $400.

The remaining 2 tickets qualify for a 5% discount. 5% of $40 is $2. So, each of these 2 tickets costs $40 - $2 = $38.

The total cost for the remaining 2 tickets is 2 * $38 = $76.

So, the total cost of all the tickets is the sum of the cost of the first 10 tickets and the cost of the remaining 2 tickets, which is $400 + $76 = $476.
</reasoning>
<answer>
$476
</answer> 
Extracted:
$476


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.151833,0.190207,179.333344,0.000000,-0.151833,0.000000,0.000000,0.000000,0.000000
2,0.000000,-0.017333,0.175934,200.000000,0.000000,-0.017333,0.000000,0.000000,0.000000,0.000000
3,0.000000,-0.107000,0.239992,182.166672,0.000005,-0.107000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.071833,0.261882,188.833344,0.000005,-0.071833,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.025667,0.062870,123.000000,0.000007,0.025667,0.000000,0.000000,0.000000,0.000000
6,0.000000,-0.072167,0.176772,196.500000,0.000004,-0.072167,0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.092833,0.191302,188.166672,0.000007,-0.092833,0.000000,0.000000,0.000000,0.000000
8,0.000000,-0.072000,0.172274,138.666672,0.000016,-0.072000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.642500,1.217141,174.333344,0.000009,-0.190833,0.000000,0.000000,0.166667,0.666667
10,0.000000,0.104167,0.051031,200.000000,0.000007,0.104167,0.000000,0.000000,0.000000,0.000000


串流輸出內容已截斷至最後 5000 行。
</reasoning>
<answer>
First, let's find out how many quarters 35 dollars is equal to. Since one quarter is $0.25, we can divide 35 dollars by 0.25.
35 / 0.25 = 140 quarters.

Next, we subtract 140 quarters (since she spent 35 dollars worth of quarters) from the initial 160 quarters.
160 - 140 = 20 quarters.

</answer> 
Extracted:
First, let's find out how many quarters 35 dollars is equal to. Since one quarter is $0.25, we can divide 35 dollars by 0.25.
35 / 0.25 = 140 quarters.

Next, we subtract 140 quarters (since she spent 35 dollars worth of quarters) from the initial 160 quarters.
160 - 140 = 20 quarters.
-------------------- Question:
John has a very inefficient toilet that uses 5 gallons of water per flush and his household flushed 15 times per day.  He replaced it with a toilet that uses 80% less water per flush.  How much water did John save in June? 
Answer:
1800 
Response:
Reasoning:
First, calculate the water usage of the new toilet per flush, then mul

TrainOutput(global_step=250, training_loss=0.000514115566927861, metrics={'train_runtime': 5250.6759, 'train_samples_per_second': 0.048, 'train_steps_per_second': 0.048, 'total_flos': 0.0, 'train_loss': 0.000514115566927861})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.09s/it, est. speed input: 9.53 toks/s, output: 36.91 toks/s]


"Calculating pi is an ongoing process as it has an infinite number of digits. However, I can provide you with the first few digits of pi.\n\nThe current record for the most accurate calculation of pi is held by the Great Internet Mersenne Initiative (GIMI), which has calculated pi to over 31.4 trillion digits. However, I won't be calculating that many digits here.\n\nLet's calculate pi to 10 digits for you:\n\npi = 3.14159265359\n\nKeep in mind that pi is a transcendental number, which means it's an irrational number that cannot be expressed as a finite decimal or fraction. The exact value of pi goes on forever without repeating.\n\nWould you like me to calculate more digits?"

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.97s/it, est. speed input: 8.76 toks/s, output: 36.33 toks/s]


"<reasoning>\nPi (π) is an irrational number, which means it cannot be expressed as a finite decimal or fraction. The value of pi is approximately equal to the ratio of a circle's circumference to its diameter. The mathematical formula for pi is π = C / D, where C is the circumference of a circle and D is the diameter of the circle.\n\nTo calculate pi, we can use a variety of mathematical methods such as:\n\n1. Leibniz formula: π = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)\n2. Gregory's series: π = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)\n3. Archimedes' method: π ≈ 3.14159 (approximation using the area and circumference of a circle)\n\nUsing these methods, we can calculate pi to a high degree of accuracy. However, the actual value of pi is an infinite, non-repeating decimal.\n\n</reasoning>\n<answer>\n3.14159 (or to many decimal places, but the actual value is an infinite, non-repeating decimal)</answer>"

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!


[unsloth](https://unsloth.ai/)

[join our discord](https://discord.gg/unsloth)

[Documentation](https://docs.unsloth.ai/)


  Join Discord if you need help +  <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i>



In [ ]:
from google.colab import files

model.save_pretrained("model0")
tokenizer.save_pretrained("model0")
!zip -r model0.zip model0
files.download("model0.zip")

updating: model0/ (stored 0%)
updating: model0/README.md (deflated 66%)
updating: model0/adapter_model.safetensors (deflated 8%)
updating: model0/adapter_config.json (deflated 56%)
updating: model0/tokenizer_config.json (deflated 94%)
updating: model0/special_tokens_map.json (deflated 71%)
updating: model0/tokenizer.json (deflated 85%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained_merged("model1", tokenizer, save_method="merged_16bit")
!zip -r model1.zip model1
files.download("model1.zip")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 1.23 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 14/28 [00:01<00:00, 18.18it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [01:39<00:00,  3.57s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model1/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model1/pytorch_model-00002-of-00002.bin...
Done.
  adding: model1/ (stored 0%)
  adding: model1/generation_config.json (deflated 28%)
  adding: model1/pytorch_model.bin.index.json (deflated 96%)
  adding: model1/config.json (deflated 52%)
  adding: model1/tokenizer_config.json (deflated 94%)
  adding: model1/special_tokens_map.json (deflated 71%)
  adding: model1/tokenizer.json (deflated 85%)
  adding: model1/pytorch_model-00001-of-00002.bin (deflated 21%)
  adding: model1/pytorch_model-00002-of-00002.bin (deflated 22%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
# 存儲 LoRA 微調權重
model.save_lora("model1_lora")

# 壓縮 LoRA 權重
!zip -r model1_lora.zip model1_lora

# 下載 LoRA 權重
files.download("model1_lora.zip")


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

  adding: model1_lora/ (stored 0%)
  adding: model1_lora/README.md (deflated 66%)
  adding: model1_lora/adapter_model.safetensors (deflated 12%)
  adding: model1_lora/adapter_config.json (deflated 56%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
import os

# 1. 掛載 Google Drive
drive.mount('/content/drive')

# 設定 Google Drive 儲存目錄
drive_path = "/content/drive/MyDrive/z-發布"

# 確保 z-發布 目錄存在
os.makedirs(drive_path, exist_ok=True)

#  2. 儲存原始模型 (`model0`)
print(" 儲存原始模型 model0 ...")
#model.save_pretrained("model0")
#tokenizer.save_pretrained("model0")
!zip -r model0.zip model0
!mv model0.zip {drive_path}/model0.zip
print(f" model0.zip 已存入 Google Drive: {drive_path}/model0.zip")

# 3️. 儲存微調後的完整模型 (`model1`)
print(" 儲存微調後的完整模型 model1 ...")
#model.save_pretrained_merged("model1", tokenizer, save_method="merged_16bit")
#!zip -r model1.zip model1
!mv model1.zip {drive_path}/model1.zip
print(f" model1.zip 已存入 Google Drive: {drive_path}/model1.zip")

# 4️. 儲存 LoRA 權重 (`model1_lora`)
print(" 儲存 LoRA 權重 model1_lora ...")
#model.save_lora("model1_lora")
#!zip -r model1_lora.zip model1_lora
!mv model1_lora.zip {drive_path}/model1_lora.zip
print(f" model1_lora.zip 已存入 Google Drive: {drive_path}/model1_lora.zip")

print("\n 所有模型已成功儲存到 Google Drive！")
